# Lecture Sensorik - Filter
*HS-Kempten FA204 WS 2018/2019 © R. Aue*

**ToC:**

1. <a href='#LowPassFilter'>Low Pass</a>  
1.1 Time Domain     
1.1.1 Convolution Example (s. scipy doc)  
1.1.2 Convolution of 2 sine signals  
1.2 Frequency Domain  
1.2.1 Frequency Response  
2. <a href='#HighPassFilter'>High Pass</a>  
2.1 Time Domain  
2.2 Frequency Domain  
3. <a href='#BandPassBandStop'>Bandpass/stop</a>  
4. <a href='#DigitalFilter'>Digital Filter</a>
5. <a href='#MatchedFilter'>Matched Filter</a>

**References:**  
https://tomroelandts.com/articles/how-to-create-a-simple-low-pass-filter


## <a id='LowPassFilter'>1. Low Pass Filter</a>

### 1.1 Time Domain 

$x (t) = \frac{\alpha}{2 \cdot \pi}$

#### 1.1.1 Example 1 (s. scipy docs)

In [ ]:
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# set signals:
sig = np.repeat([0., 1., 0.], 100)
win = signal.hann(50)
filtered = signal.convolve(sig, win, mode='same') / sum(win)

# plot:
fig, (ax_orig, ax_win, ax_filt) = plt.subplots(3, 1, sharex=True)
ax_orig.plot(sig)
ax_orig.set_title('Original pulse')
ax_orig.margins(0, 0.1)
ax_win.plot(win)
ax_win.set_title('Filter impulse response')
ax_win.margins(0, 0.1)
ax_filt.plot(filtered)
ax_filt.set_title('Filtered signal')
ax_filt.margins(0, 0.1)
fig.tight_layout()


#### 1.1.2 Example 2  

${ \displaystyle y(t) = s(t) * h(t) }$
- create input signal $s(t)$ as sum of two independent signals w/ different frequencies and amplitudes;
- create impulse response $h(t)$ of ideal cutoff filter (i.e. using sinc-function)  
- apply window to impulse response, because  sinc has infinite length; simple truncation would lead to massive ripple;  
- convolute signal w/ filter impulse response;  

In [ ]:
import numpy as np
import scipy.constants as spc  # e.g. speed of light
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline --no-import-all

# consts:
f1 = 1.0   # input signal 1 frequency [Hz]
a1 = 2.0   # input signal 1 amplitude [unit]
f2 = 3.0   # input signal 2 frequency [Hz]
a2 = 1.0   # input signal 2 amplitude [unit]
f_sample = 10.0   # sampling rate  [Hz] (i.e. >= 2*f)
f_cutoff = 2.5   # cutoff frequency [Hz]: must be < than f_sample/2;
bt = 0.2  # transition bandwidth [Hz]

# number of samples resp. length of impulse response:
N = int(np.ceil((4 / (bt/f_sample))))
if not N % 2: N += 1  # make sure that N is odd (i.e. symmetric).

n = np.arange(N)
t = np.arange(0,N/f_sample,1/f_sample)

# create input signal:
s = a1 * np.sin(2 * np.pi * f1 * t) + a2 * np.sin(2 * np.pi * f2 * t)

# Compute sinc filter.
h = np.sinc(2 * (f_cutoff/f_sample) * (n - (N - 1) / 2.))
 
# Compute Blackman window.
w = np.blackman(N)
 
# apply window.
hw = h * w
 
# Normalize to get unity gain; for low pass sum of all coeeficients must be 1;
hw = hw / np.sum(hw)

# apply the low pass by doing the convolution:
y = np.convolve(s, hw)


# Compute frequency response; only keep first half.
H = np.abs(np.fft.fft(hw))[0 : N // 2 + 1]

# plot signals:
plt.figure(figsize=(14,7))
plt.subplot(221)
plt.title('Input Signals s1(t) + s2(t)')
plt.plot(t,s, 'b', label='S1 + S2')
plt.grid(True, which='both')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude [Unit]')
plt.legend()

plt.subplot(222)
plt.title('Windowed Low Pass Filter Impulse Response h(t)')
plt.plot(hw)
plt.grid(True, which='both')
plt.xlabel('# [n]')
plt.ylabel('Amplitude [Unit]')

plt.subplot(223)
plt.title('Low Pass Filtered Output Signal y(t)')
plt.plot(t[0:N-1],y[int(len(y)/4):int(len(y)*3/4)])
plt.grid(True, which='both')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude [Unit]')

plt.subplot(224)
plt.title('Low Pass Frequency Response')
plt.plot(np.linspace(0, 0.5, len(H))*f_sample, 20 * np.log10(H))
plt.xlabel('Frequency [Hz]')
plt.ylabel('Gain [dB]')
plt.ylim([-120, 10])
plt.grid()

plt.tight_layout()
plt.show()


### 1.2 Frequency Domain

### 1.2.1 Frequency Response

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
fc = 0.2  # Cutoff frequency as a fraction of the sampling rate (in (0, 0.5)).
N = 50    # Number of coefficients.
L = 2048  # Length of frequency response.
 
# Compute sinc filter with Hamming window.
n = np.arange(N)
h = np.sinc(2 * fc * (n - (N - 1) / 2.)) * np.hamming(N)
h /= np.sum(h)
 
# Pad filter with zeros.
h_padded = np.zeros(L)
h_padded[0 : N] = h
 
# Compute frequency response; only keep first half.
H = np.abs(np.fft.fft(h_padded))[0 : L // 2 + 1]
 
# Plot frequency response (in dB).
plt.figure()
plt.plot(np.linspace(0, 0.5, len(H)), 20 * np.log10(H))
plt.xlabel('Low Pass Normalized frequency')
plt.ylabel('Gain [dB]')
plt.ylim([-100, 10])
plt.grid()
plt.show()


## <a id='HighPassFilter'>2. High Pass Filter</a>

### 2.1 Time Domain  

Define impulse response of high pass:
- create a low-pass filter (i.e. using sinc function)  
- apply spectral inversion to low pass filter impulse response h[n]:  
  -   Change the sign of each value in h[n]   
  -   Add one to the value in the center.

In [ ]:
import numpy as np
import scipy.constants as spc  # e.g. speed of light
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline --no-import-all

# consts:
f1 = 1.0   # input signal 1 frequency [Hz]
a1 = 2.0   # input signal 1 amplitude [unit]
f2 = 4.0   # input signal 2 frequency [Hz]
a2 = 1.0   # input signal 2 amplitude [unit]
f_sample = 20.0   # sampling rate  [Hz] (i.e. >= 2*f)
f_cutoff = 3   # cutoff frequency [Hz]: must be < than f_sample/2;
bt = 0.2  # transition bandwidth [Hz]

# number of samples resp. length of impulse response:
N = int(np.ceil((4 / (bt/f_sample))))
if not N % 2: N += 1  # make sure that N is odd (i.e. symmetric).

    
n = np.arange(N)
t = np.arange(0,N/f_sample,1/f_sample)

# create input signal:
s = a1 * np.sin(2 * np.pi * f1 * t) + a2 * np.sin(2 * np.pi * f2 * t)

# Compute sinc filter.
h = np.sinc(2 * (f_cutoff/f_sample) * (n - (N - 1) / 2.))
 
# Compute Blackman window.
w = np.blackman(N)
 
# apply window.
hw = h * w
 
# Normalize to get unity gain.
hw = hw / np.sum(hw)

# Create a high-pass filter from the low-pass filter through spectral inversion.
hw = -hw
hw[int((N - 1) / 2)] += 1


# apply the low pass by doing the convolution:
y = np.convolve(s, hw)


# Compute frequency response; only keep first half.
H = np.abs(np.fft.fft(hw))[0 : N // 2 + 1]

# plot signals:
plt.figure(figsize=(14,10))
plt.subplot(411)
plt.title('Input Signals s1(t) + s2(t)')
plt.plot(t, s, 'b', label='S1 + S2')
plt.grid(True, which='both')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude [Unit]')
plt.legend()

plt.subplot(412)
plt.title('Windowed High Pass Filter Impulse Response h(t)')
plt.plot(hw)
plt.grid(True, which='both')
plt.xlabel('# [n]')
plt.ylabel('Amplitude [Unit]')

plt.subplot(413)
plt.title('High Pass Filtered Output Signal y(t)')
plt.plot(t[0:N-1],y[int(len(y)/4):int(len(y)*3/4)])
plt.grid(True, which='both')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude [Unit]')

plt.subplot(414)
plt.title('High Pass Frequency Response')
plt.plot(np.linspace(0, 0.5, len(H))*f_sample, 20 * np.log10(H))
plt.xlabel('Frequency [Hz]')
plt.ylabel('Gain [dB]')
plt.ylim([-120, 10])
plt.grid()

plt.tight_layout()
plt.show()

### 2.2 Frequency Domain

#### 2.2.1 Frequency Response

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
fc = 0.2  # Cutoff frequency as a fraction of the sampling rate (in (0, 0.5)).
N = 25    # Number of coefficients.
L = 1024  # Length of frequency response.
 
# Compute sinc filter with Hamming window.
n = np.arange(N)
h = np.sinc(2 * fc * (n - (N - 1) / 2.)) * np.hamming(N)
h /= np.sum(h)

# Create a high-pass filter from the low-pass filter through spectral inversion.
h = -h
h[int((N - 1) / 2)] += 1
 
# Pad filter with zeros.
h_padded = np.zeros(L)
h_padded[0 : N] = h
 
# Compute frequency response; only keep first half.
H = np.abs(np.fft.fft(h_padded))[0 : L // 2 + 1]
 
# Plot frequency response (in dB).
plt.figure()
plt.plot(np.linspace(0, 0.5, len(H)), 20 * np.log10(H))
plt.xlabel('High Pass Normalized Frequency')
plt.ylabel('Gain [dB]')
plt.ylim([-100, 10])
plt.grid()
plt.show()

## <a id='BandPassBandStop'>3. Band Pass / Band Stop</a>  
- combination of Low- and High-Pass filters

In [ ]:
# ...

### Further Exercises:
1. Define Bandpass filter based on given examples of low and high pass filters

## <a id='DigitalFilter'>4. Digital Filter</a>

### 4.1 FIR Low Pass - $H(z) = f(Impulse Response)$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
 
# consts:
fc = 0.2                # Cutoff frequency as a fraction of the sampling rate (in (0, 0.5)).
N = [5,10,20,50,125]    # Number of filter coefficients.
L = 4096                # Length of frequency response.
colors = cm.rainbow(np.linspace(0, 1, len(N)))

plt.figure(figsize=(8,5))
# Compute sinc filter with Hamming window.
for N_,c in zip(N, colors):
    n = np.arange(N_)
    h = np.sinc(2 * fc * (n - (N_ - 1) / 2.)) * np.hamming(N_)
    h /= np.sum(h)
    
    # Pad filter with zeros.
    h_padded = np.zeros(L)
    h_padded[0 : N_] = h
 
    # Compute frequency response; only keep first half.
    H = np.abs(np.fft.fft(h_padded))[0 : L // 2 + 1]
    
    plt.plot(np.linspace(0, 0.5, len(H)), 20 * np.log10(H), color=c, label='N = ' + str(N_))
 

# Plot frequency response (in dB)
plt.title('FIR Low Pass Gain=f(N)')
plt.xlabel('Normalized frequency')
plt.ylabel('Gain [dB]')
plt.ylim([-100, 10])
plt.legend()
plt.grid()
plt.show()

### 4.2 FIR Low Pass - Remez Algorithm   
http://www.ee.iitm.ac.in/~nitin/teaching/ee5480/fir

In [ ]:
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

# consts: 
N = [5,10,20,50,125]  # filter order list;
colors = cm.rainbow(np.linspace(0, 1, len(N)))

plt.figure(figsize=(8,5))
# define FIR based on Remez
for N_, c in zip(N, colors):
    # remez(numtaps, bands, desired, weight=None, Hz=None, type='bandpass', maxiter=25, grid_density=16, fs=None)
    lpf = signal.remez(N_, [0, 0.2, 0.3, 0.5], [1.0, 0.0])

    w, h = signal.freqz(lpf) # calc frequency response #freqz(b, a=1, worN=None, whole=False, plot=None)[source]
    # print(lpf)
    plt.plot(w/(2*np.pi), 20*np.log10(abs(h)), color=c, label='N = ' + str(N_))


plt.title('FIR Low Pass Gain=f(N) - Remez Algorithm')
plt.xlabel('Normalized frequency')
plt.ylabel('Gain [dB]')
plt.ylim([-100, 10])
plt.legend()
plt.grid()
plt.show()

### 4.3 IIR/FIR Filter Design   
- iirfilter() => gain = f(N)
- iirdesign() => N = gain(frequency)

In [ ]:
from scipy import signal
from scipy.signal import remez
from scipy.signal import freqz
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

# consts: 
N = [2,5,10,20,30]  # filter order list;
colors = cm.rainbow(np.linspace(0, 1, len(N)))

plt.figure(figsize=(10,6))

# design w iirfilter; gain=f(N)
plt.subplot(121)
for N_, c in zip(N, colors):
    b, a = signal.iirfilter(N_, 0.2,  btype='lowpass', ftype='butter')
    w, h = freqz(b, a) # calc frequency response #freqz(b, a=1, worN=None, whole=False, plot=None)
    plt.plot(w/(2*np.pi), 20*np.log10(abs(h)), color=c, label='N = ' + str(N_))

plt.title('Butterworth IIR/FIR - IIRFILTER() - Low Pass Gain=f(N)')
plt.xlabel('Normalized frequency')
plt.ylabel('Gain [dB]')
plt.ylim([-100, 10])
plt.legend()
plt.grid()



# design w/ iirdesign(): N=f(f_p, f_s, r_p, a_s)
f_p = 0.270   # end of passband, normalized frequency
f_s = 0.352   # start of the stopband, normalized frequency 
r_p = 0.1     # passband maximum loss [dB] (gpass)
a_s = 100      # stoppand min attenuation [dB] (gstop)

plt.subplot(122)
b, a = signal.filter_design.iirdesign(f_p, f_s, r_p, a_s, ftype='butter')
w, h = signal.freqz(b, a) # calc frequency response #freqz(b, a=1, worN=None, whole=False, plot=None)
plt.plot(w/(2*np.pi), 20*np.log10(abs(h)), label='N = ' + str(len(a)))

plt.title('Butterworth IIR/FIR - IIRDESIGN() - Low Pass Gain=f(N)')
plt.xlabel('Normalized frequency')
plt.ylabel('Gain [dB]')
plt.ylim([-100, 10])
plt.legend()
plt.grid()


plt.tight_layout()
plt.show()

## <a id='MatchedFilter'>5. Matched Filter</a>

### 5.1 Scipy Example
s. https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.correlate.html

In [ ]:
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np

# create signals:
sig = np.repeat([0., 1., 1., 0., 1., 0., 0., 1.], 128)
sig_noise = sig + np.random.randn(len(sig))

#
corr = signal.correlate(sig_noise, np.ones(128), mode='same') / 128

clock = np.arange(64, len(sig), 128)

# plot:
fig, (ax_orig, ax_noise, ax_corr) = plt.subplots(3, 1, sharex=True)
fig.set_size_inches(12, 8)
ax_orig.plot(sig)
ax_orig.plot(clock, sig[clock,], 'ro')
ax_orig.set_title('Original signal')
ax_noise.plot(sig_noise)
ax_noise.set_title('Signal with noise')
ax_corr.plot(corr)
ax_corr.plot(clock, corr[clock,], 'ro')
ax_corr.axhline(0.5, ls=':')
ax_corr.set_title('Cross-correlated with rectangular pulse')
ax_orig.margins(0, 0.1)
fig.tight_layout()

# Note: ignore '... signaltools.py:491: FutureWarning ...' when running script first time, because it's a know scipy bug;just run it again;

### 5.2 Sinusoidal signal with Gaussian noise with added 

In [ ]:
"""
"""
from scipy import signal
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline --no-import-all

# consts:
fs = 4096           # sampling rate [Hz]  
T = 4               # duration [s]
amp = 0.1           # amplitude of the sinusoid 
fsin = 13            # frequency of the signal 

N = T*fs            # total number of points 

# time interval spaced with 1/fs 
t = np.arange(0, T, 1./fs)
 
# white noise: 
noise = np.random.normal(size=t.shape)

# sinusoidal signal with amplitude amp:
template = amp*np.sin(fsin*2*np.pi*t)
 
# data: signal (template) with added noise 
data = template + noise

# plot:
plt.figure()
plt.figure(figsize=(14,7))
plt.title('Sinusoidal signal with Gaussian noise with added')
plt.plot(t, data, '-', color="grey", label='Signal + Noise')
plt.plot(t, template, '-', color="green", linewidth=2, label='Signal')
plt.xlim(0, T)
plt.xlabel('time [s]')
plt.ylabel('data = signal + noise')
plt.grid(True)
plt.legend()
plt.show()
# plt.savefig("gaussian_noise_w_signal.png", format="png", bbox_inches="tight")


**Fourier transform of the signals:**

In [ ]:
"""
Discrete Fourier transform of the signal in noise 
"""

# FFT of the template and the data (not normalized)  
template_fft = np.fft.fft(template)
data_fft = np.fft.fft(data)

# Sampling frequencies up to the Nyquist limit (fs/2)  
sample_freq = np.fft.fftfreq(t.shape[-1], 1./fs)

# plot:
max_f = int(len(data_fft)/2)
plt.figure()
plt.figure(figsize=(10,5))
plt.plot(sample_freq[:max_f], np.abs(data_fft[:max_f])/np.sqrt(fs), color="grey", label='Signal + Noise')
plt.plot(sample_freq[:max_f], np.abs(template_fft[:max_f])/np.sqrt(fs), color="red", alpha=0.5, linewidth=4, label='Signal')

# taking positive spectrum only: plt.xlim(0, np.max(sample_freq)) 
plt.title('DFT of Signal and Noise')
plt.xlim(0, 4 * fsin)
plt.xlabel('Frequency [Hz]')
plt.ylabel('Power')
plt.grid(True)
plt.legend()
plt.show()


#### Exercise
**Create and apply Matched Filter:**
1. DFT(original signal)
2. Conjugate complex of 1.
3. Apply filter
4. Inverse DFT of filtered signal

In [ ]:
"""
Matched filter 
"""
# conjugate complex of original signal i.e. the matched filter:
matched_filter=np.conj(template_fft)

# apply filter:
signal_f=matched_filter*data_fft

# transform to time domain:
signal_t=2*np.fft.ifft(signal_f)/N

# plot:
plt.figure(figsize=(12,6))
plt.plot(t[:int(N/5)], template[:int(N/5)], '-', color="green", linewidth=2, label='Signal org')
plt.plot(t[:int(N/5)],np.real(10*signal_t[:int(N/5)]), '-', color="red", linewidth=2, label='Signal recovered')
plt.grid(True)
plt.legend()
plt.show()